# 제목
최근 유행하는 치즈케이크 집의 분포를 알아보자

## 1. 사전 환경 설정

In [5]:
from selenium import webdriver
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd 
import re

## 2. RAW 데이터 수집

### (1) 필요한 사전정보 입력

In [5]:
handle = '' #인스타그램 아이디
pwd = ''  #인스타그램 비밀번호

keyword= '치즈케이크'   #검색할 키워드
num_of_pages = 800 #몇 번 스크롤 할 건지; e.g. 200 --> 1730개 게시글 긁어짐

### (2) 크롤링 관련 함수 정의
* 1단계 : 검색 페이지에서 글 URL 가져오기

In [7]:
def InstagramUrlFromKeyword (browser,keyword,num_of_pagedowns):
    keyword_url_encode=quote(keyword)
    url='https://www.instagram.com/explore/tags/'+keyword+'/?hl=ko'
    browser.get(url)
    time.sleep(3)
    login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
    login.click()
    time.sleep(3)
    user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
    pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
    user_id.send_keys(handle)
    pw.send_keys(pwd)
    button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
    button.click()
    time.sleep(3)
    arr_href=[]
    body=browser.find_element_by_tag_name('body')
    for i in range(num_of_pagedowns):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        post=browser.find_elements_by_class_name('v1Nh3')
        for j in post:
            href_str=j.find_element_by_css_selector('a').get_attribute('href')
            arr_href.append(href_str)
    return set(arr_href)    

char = r'#[\d\w]+'
char2 = re.compile('[^ 0-9a-zA-Zㄱ-ㅣ가-힣!#?]')

* 2단계 : 글 URL에서 필요한 데이터 수집

In [ ]:
def IdHashTagFromInstagram(browser,url):
    browser.get(url)
    datetime = browser.find_element_by_class_name('_1o9PC.Nzb55').get_attribute('title')
    href=browser.find_elements_by_class_name('C4VMK')
    #포스트 내용(글, 태그)
    total_hash_text=[]    
    for i in range(0, len(href)):
        hash_text= href[i].find_element_by_css_selector('span').text
        total_hash_text.append(hash_text)
    tags = re.findall(char, char2.sub(' ',str(total_hash_text)))
    user_name = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[1]/div[1]/h2/a').text
    like_count = int(browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/div/button/span').text)
    location = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a').text
    
    return datetime, user_name, like_count, tags, location

### (3) 크롤링 시행

In [49]:

insta_df = []
df2 = ""
c = ""


browser= webdriver.Chrome('c:/Temp/chromedriver.exe')
arr=InstagramUrlFromKeyword(browser,keyword,num_of_pages)
insta_df=pd.DataFrame(columns = {'Datetime', 'User_Name', 'Like_Count', 'Tags', 'Location'})

for url in arr:
        try:
            datetime, user_name, like_count, tags, location = IdHashTagFromInstagram(browser,url)
            insta_df = insta_df.append({'Datetime': datetime, 'User_Name':user_name, 'Like_Count':like_count,'Tags': tags, 'Location' : location}, ignore_index = True)
        except:
            continue

KeyboardInterrupt: 

In [ ]:
from collections import Counter

insta_str = []
for tags in insta_df['Tags']:
    insta_str.extend(tags)
    
c = Counter(insta_str)
c.most_common(50)

### (4) 크롤링 완료 후 데이터가 잘 수집되었는지 확인

In [112]:
len(arr)

6219

In [9]:
insta_df

,Location,Datetime,Tags,Like_Count,User_Name
0,향리단커피,2019년 12월 22일,"[#향리단스콘, #향남맛집, #스콘, #스콘맛집, #화성카페, #향남카페, #수원카...",48,hrdcoffee
1,Changwon,2019년 12월 30일,[],11,staymood_candle
2,The Cheesecake Factory,2019년 12월 17일,"[#대기20분, #줄줄이소세지, #하와이, #하와이신혼여행, #하와이맛집, #치즈케...",35,_dbddbd_
3,인사이드커피-Insidecoffee,2019년 12월 17일,"[#코흘리게, #못생긴, #나의중학생시절, #기억하는사람, #16년지기, #고니, ...",38,jojosuhyun
4,"Seoul, South Korea",2019년 12월 30일,"[#월요일, #모닝커피, #커피빈, #치즈케이크, #셋이아닌둘이서, #커피스타그램,...",21,eunyoung3280
...,...,...,...,...,...
1098,은계지구,2019년 12월 27일,"[#은계지구, #초밥, #허니콤보, #핫도그, #신전, #오돌뼈, #치즈케이크, #...",41,yang_hj2
1099,얼스어스,2019년 12월 18일,[#얼스어스],21,juny_hoy
1100,스윗레시피 Sweet Recipe,2019년 12월 14일,[#스윗레시피],148,sweet_0310
1101,하우다 과자점,2019년 12월 25일,[],25,hauda_jeju


### (5) 크롤링 결과 수집된 데이터를 csv로 저장
euc-kr로 저장하면 인코딩 문제로 오류나서 utf-8로 저장함

In [13]:
insta_df.to_csv(keyword + '_' + str(num_of_pages) + '.csv',mode='w', encoding='euc-kr')

UnicodeEncodeError: 'euc_kr' codec can't encode character '\u0e1e' in position 14: illegal multibyte sequence

In [11]:
insta_df.to_csv(keyword + '_' + str(num_of_pages) + '.csv',mode='w', encoding='UTF-8')

## 3. 데이터 처리

### (1) (참고) 시연을 위해 크롤링으로 얻은 raw data 로드

In [51]:
insta_df = pd.read_csv('C:/Users/student/Desktop/PROJECT1_dy/rawdata.csv', index_col=0, encoding='UTF-8')
insta_df

,Location,Datetime,Tags,Like_Count,User_Name,url
0,향리단커피,2019년 12월 22일,"['#향리단스콘', '#향남맛집', '#스콘', '#스콘맛집', '#화성카페', '...",48,hrdcoffee,https://www.instagram.com/p/B6XfP5YgO2g/
1,Changwon,2019년 12월 30일,[],11,staymood_candle,https://www.instagram.com/p/B6n6jBfFVNK/
2,The Cheesecake Factory,2019년 12월 17일,"['#대기20분', '#줄줄이소세지', '#하와이', '#하와이신혼여행', '#하와...",35,_dbddbd_,https://www.instagram.com/p/B6po6sxghvm/
3,인사이드커피-Insidecoffee,2019년 12월 17일,"['#코흘리게', '#못생긴', '#나의중학생시절', '#기억하는사람', '#16년...",38,jojosuhyun,https://www.instagram.com/p/B6qIMZdprLK/
4,"Seoul, South Korea",2019년 12월 30일,"['#월요일', '#모닝커피', '#커피빈', '#치즈케이크', '#셋이아닌둘이서'...",21,eunyoung3280,https://www.instagram.com/p/B6pOCC8jNaf/
...,...,...,...,...,...,...
1098,은계지구,2019년 12월 27일,"['#은계지구', '#초밥', '#허니콤보', '#핫도그', '#신전', '#오돌뼈...",41,yang_hj2,https://www.instagram.com/p/B6hlQHuJd-c/
1099,얼스어스,2019년 12월 18일,['#얼스어스'],21,juny_hoy,https://www.instagram.com/p/B6KjLwjlwss/
1100,스윗레시피 Sweet Recipe,2019년 12월 14일,['#스윗레시피'],148,sweet_0310,https://www.instagram.com/p/B6Igd7AJbZw/
1101,하우다 과자점,2019년 12월 25일,[],25,hauda_jeju,https://www.instagram.com/p/B6aBT51nBFW/


### (((((((((((((((참고) 빈도분석

In [58]:
insta_df.dtypes

Location      object
Datetime      object
Tags          object
Like_Count     int64
User_Name     object
url           object
dtype: object

In [67]:
from collections import Counter

insta_tags = []
for tags in insta_df['Tags']:
    tags = tags.strip('[').strip(']').replace("'", '')
    tag_list = tags.split(', ')
    while '' in tag_list:
        tag_list.remove('')
    insta_tags.extend(tag_list)
    
c = Counter(insta_tags)
    
c.most_common(50)

[('#치즈케이크', 871),
 ('#카페', 172),
 ('#일상', 137),
 ('#디저트', 120),
 ('#카페투어', 107),
 ('#크리스마스', 105),
 ('#cafe', 94),
 ('#카페스타그램', 84),
 ('#cheesecake', 84),
 ('#케이크', 83),
 ('#디저트카페', 80),
 ('#커피', 77),
 ('#coffee', 77),
 ('#마카롱', 76),
 ('#먹스타그램', 75),
 ('#데일리', 75),
 ('#좋아요', 63),
 ('#daily', 54),
 ('#딸기케이크', 51),
 ('#dessert', 49),
 ('#맛집', 49),
 ('#맛스타그램', 49),
 ('#아메리카노', 48),
 ('#스콘', 46),
 ('#딸기', 46),
 ('#맞팔', 46),
 ('#디저트맛집', 46),
 ('#부산카페', 44),
 ('#크리스마스케이크', 43),
 ('#소통', 42),
 ('#티라미수', 40),
 ('#마카롱맛집', 39),
 ('#cake', 39),
 ('#예쁜카페', 37),
 ('#브라우니', 34),
 ('#메리크리스마스', 33),
 ('#먹방', 33),
 ('#팔로우', 31),
 ('#초코케이크', 31),
 ('#마들렌', 30),
 ('#제리치즈케이크', 29),
 ('#선물', 29),
 ('#macarons', 29),
 ('#좋반', 29),
 ('#밀크티', 28),
 ('#커피맛집', 28),
 ('#크리스마스이브', 28),
 ('#세종시마카롱', 28),
 ('#새롬동카페', 28),
 ('#신상카페', 27)]

In [45]:
from collections import Counter

insta_str = []
for tags in insta_df['Tags']:
    insta_str.extend(tags)
    
c = Counter(insta_str)
c.most_common(50)

'''
df2 = pd.DataFrame(columns = {'word', 'count'})
for word in c:
    df2 = df2.append({'word':word, 'count':c[word]}, ignore_index = True)

'''


c.most_common(50)
#결과 저장
#df2.to_csv('test_' + 'freq' + '.csv',mode='w', encoding='UTF-8')

[("'", 30066),
 ('#', 15033),
 (',', 14075),
 (' ', 14075),
 ('카', 2942),
 ('이', 2714),
 ('크', 2436),
 ('페', 2385),
 ('e', 2063),
 ('케', 1945),
 ('스', 1887),
 ('a', 1459),
 ('치', 1373),
 ('즈', 1353),
 ('o', 1108),
 ('[', 1103),
 (']', 1103),
 ('리', 1056),
 ('c', 1028),
 ('맛', 922),
 ('마', 901),
 ('트', 857),
 ('f', 834),
 ('s', 812),
 ('집', 808),
 ('r', 742),
 ('동', 720),
 ('t', 675),
 ('타', 627),
 ('그', 613),
 ('l', 612),
 ('i', 597),
 ('라', 589),
 ('아', 541),
 ('디', 524),
 ('일', 514),
 ('램', 503),
 ('산', 476),
 ('피', 475),
 ('저', 460),
 ('n', 458),
 ('주', 454),
 ('기', 448),
 ('대', 447),
 ('커', 444),
 ('롱', 444),
 ('d', 419),
 ('수', 413),
 ('k', 412),
 ('h', 401)]

### (2) 지도 API를 이용하여 위치정보 텍스트를 위도, 경도로 변환